# Gibbs free energy potential minimization (T, P)
### With rhyolite-MELTS and $\mu$O<sub>2</sub> constrained using the empirical method of Kress and Carmichael (1991)

This notebook demonstates the use of the Equilibrate class in the equilibrate module to perform equilibrium crystallization clculations for the special case of oxygen fugacity constrained using the model of Kress and Carmichael (1991).  This model is applicable to natural composition silicate liquids and is usually applied in conjunction with thermodynamic models published by Ghiorso and Sack (1995) [MELTS], Ghiorso et al., (2003) [pMELTS], Gualda et al. (2012) [rhyolite-MELTS] or Ghiorso and Gualda (2015) [rhyoliteMELTS + CO2].

**References:**  

Ghiorso, Mark S., and Sack, Richard O. (1995) Chemical Mass Transfer in Magmatic Processes. IV. A Revised and Internally Consistent Thermodynamic Model for the Interpolation and Extrapolation of Liquid-Solid Equilibria in Magmatic Systems at Elevated Temperatures and Pressures. Contributions to Mineralogy and Petrology, 119, 197-212  

Ghiorso, Mark S., Hirschmann, Marc M., Reiners, Peter W., and Kress, Victor C. III (2002) The pMELTS: An revision of MELTS aimed at improving calculation of phase relations and major element partitioning involved in partial melting of the mantle at pressures up to 3 GPa. Geochemistry, Geophysics, Geosystems 3(5), 10.1029/2001GC000217  

Gualda G.A.R., Ghiorso M.S., Lemons R.V., Carley T.L. (2012) Rhyolite-MELTS: A modified calibration of MELTS optimized for silica-rich, fluid-bearing magmatic systems. Journal of Petrology, 53, 875-890  

Ghiorso M.S., Gualda, G.A.R., (2015) An H<sub>2</sub>O-CO<sub>2</sub> mixed fluid saturation model compatible with rhyolite-MELTS. Contributions to Mineralogy and Petrology 2015, doi:10.1007/s00410-015-1141-8  

Kress and Carmichael (1991, The compressibility of silicate liquids containing Fe<sub>2</sub>O<sub>3</sub> and the effect of composition, temperature, oxygen fugacity and pressure on their redox states, Contributions to Mineralogy and Petrology 108:82-92

**Note** that the method demonstrated in this notebook should ideally be cast into a proper Khorzhinskii potential minimization, as is done for the $\mu$H<sub>2</sub>O constraint method illustrated in the next notebook. However, the method of Kress and Carmichael (1991) is inconsistent with the thermodynamic models of Ghiorso and others, even though the latter were dependently calibrated on the basis of the former. This inconsistency prevents a minima in the Khorzhinskii potential of a system containing a Ghiorso-like liquid model from reproducing the calibration of Kress and Carmichael. Hence, the empirical method of this notebook is made avaialble for practical application; the method used here mimics the method used in MELTS, etc.

In [ ]:
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
import sys

In [ ]:
from thermoengine import core, phases, model, equilibrate
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## Create phases for equilibrium assemblages
Minimal rhyolitic phase assemblage

In [ ]:
src_obj = core.get_src_object('EquilibrateUsingMELTSv102')
modelDB = model.Database(liq_mod='v1.0')

In [ ]:
Liquid = modelDB.get_phase('Liq')
Feldspar = modelDB.get_phase('Fsp')
Quartz = modelDB.get_phase('Qz')
Spinel = modelDB.get_phase('SplS')
Opx = modelDB.get_phase('Opx')
RhomOx = modelDB.get_phase('Rhom')

The Berman model database provides the SWIM water model by default.  Instead, override that choice by instantiating the MELTS 1.0.2 water model directly.

In [ ]:
Water = phases.PurePhase('WaterMelts', 'H2O', calib=False)

## Define elements in system and phases in system
These 15 elements are required for the silicate liquid model of MELTS (rhyolite-MELTS)

In [ ]:
elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
phs_sys = [Liquid, Feldspar, Water, Quartz, Spinel, Opx, RhomOx]

## Composition of the system
This is a high-silica rhyolite

In [ ]:
grm_oxides = {
    'SiO2':  77.5, 
    'TiO2':   0.08, 
    'Al2O3': 12.5, 
    'Fe2O3':  0.207,
    'Cr2O3':  0.0, 
    'FeO':    0.473, 
    'MnO':    0.0,
    'MgO':    0.03, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    0.43, 
    'Na2O':   3.98, 
    'K2O':    4.88, 
    'P2O5':   0.0, 
    'H2O':    5.5
}
tot_grm_oxides = 0.0
for key in grm_oxides.keys():
    tot_grm_oxides += grm_oxides[key]

Cast this composition as moles of elements for input to the Equilibrate class

In [ ]:
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
if not Liquid.test_endmember_comp(moles_end):
    print ("Calculated composition is infeasible!")
mol_elm = Liquid.covert_endmember_comp(moles_end,output='moles_elements')

In [ ]:
blk_cmp = []
for elm in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(elm)
    blk_cmp.append(mol_elm[index])
blk_cmp = np.array(blk_cmp)

# Run the calculation

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

Uncomment here and in the next cell to generate a call graph that traces execution times by method and module 

In [ ]:
#%load_ext snakeviz

### For reference, equilibrate at an initial temperature and pressure

In [ ]:
#%%snakeviz --new-tab
t = 1034.0
p = 1750.0
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0, stats=False)
state.print_state()

Compute the system oxygen fugacity using Kress and Carmichael (1991) along with the total moles of oxygen in the system

In [ ]:
results = []
results.append((t-273.15,p/10.0) + equil.kc_print_state(state))

### Assign an oxygen buffer, and recalculate ...

In [ ]:
NNO_offset = 0.0
state = equil.execute(t, p, bulk_comp=blk_cmp, con_deltaNNO=NNO_offset, debug=0, stats=True)
state.print_state()

In [ ]:
results.append((t-273.15,p/10.0) + equil.kc_print_state(state))

### Next, loop through a series of temperatures

In [ ]:
deltaT = 1
state = equil.execute(t-float(deltaT), p, state=state, con_deltaNNO=NNO_offset, debug=0, stats=True)
state.print_state()
results.append((t-float(deltaT)-273.15,p/10.0) + equil.kc_print_state(state))

In [ ]:
deltaT = 2
state = equil.execute(t-float(deltaT), p, state=state, con_deltaNNO=NNO_offset, debug=0, stats=True)
state.print_state()
results.append((t-float(deltaT)-273.15,p/10.0) + equil.kc_print_state(state))

In [ ]:
deltaT = 3
state = equil.execute(t-float(deltaT), p, state=state, con_deltaNNO=NNO_offset, debug=0, stats=True)
state.print_state()
results.append((t-float(deltaT)-273.15,p/10.0) + equil.kc_print_state(state))

In [ ]:
deltaT = 4
state = equil.execute(t-float(deltaT), p, state=state, con_deltaNNO=NNO_offset, debug=0, stats=True)
state.print_state()
results.append((t-float(deltaT)-273.15,p/10.0) + equil.kc_print_state(state))

In [ ]:
deltaT = 5
state = equil.execute(t-float(deltaT), p, state=state, con_deltaNNO=NNO_offset, debug=0, stats=True)
state.print_state()
results.append((t-float(deltaT)-273.15,p/10.0) + equil.kc_print_state(state))

# Plot results of the sequence
Plot total oxygen content of the system against temperature

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
x = []
y = [] 
for t,p,log10fO2,log10NNO,moles_O in results:
    x.append(t)
    y.append(moles_O)
plt.plot(np.array(x), np.array(y), 'r-')
plt.xlabel('T °C')
plt.ylabel('moles of O')
plt.show()